First, we import our dependencies:

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import psycopg as pg
from dotenv import load_dotenv
import pandas as pd
import os

Now we load our environment variables from `.env`

In [2]:
load_dotenv()

True

Now, we connect to the database:

In [3]:
# The CONNECTION_STRING is defined in the `.env` file
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')

connection_string = f"dbname='{database}' host='{host}' port={port} user='{user}' password='{password}'"

conn = pg.connect(connection_string)
cur = conn.cursor()

We create an interactive UI for viewing samples of the database and exporting its data:

In [4]:
def show_main_menu():
    clear_output(wait=True)
    display(menu_box)

def show_category_submenu(selected_category=None):
    clear_output(wait=True)
    cur.execute("SELECT DISTINCT category FROM food_inventory ORDER BY category;")
    categories = [row[0] for row in cur.fetchall()]
    cat_buttons = [widgets.Button(description=cat) for cat in categories]
    return_btn = widgets.Button(description='Return', button_style='danger')

    def make_cat_handler(category):
        def handler(b):
            show_category_submenu(selected_category=category)
        return handler

    for btn, cat in zip(cat_buttons, categories):
        btn.on_click(make_cat_handler(cat))
    def on_return_clicked(b):
        show_sample_submenu()
    return_btn.on_click(on_return_clicked)
    display(widgets.VBox(cat_buttons + [return_btn]))
    # Display the DataFrame outside the widget area if a category is selected
    if selected_category is not None:
        cur.execute("SELECT * FROM food_inventory WHERE category = %s", (selected_category,))
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        display(df)

def show_sample_submenu():
    clear_output(wait=True)
    random_btn = widgets.Button(description='View random sample')
    bycat_btn = widgets.Button(description='View by category')
    expiring_btn = widgets.Button(description='View expiring soon')
    return_btn = widgets.Button(description='Return', button_style='danger')

    def on_random_clicked(b):
        clear_output(wait=True)
        cur.execute("SELECT * FROM food_inventory ORDER BY RANDOM() LIMIT 5;")
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        display(widgets.VBox([random_btn, bycat_btn, expiring_btn, return_btn]))
        display(df)

    def on_bycat_clicked(b):
        show_category_submenu()

    def on_expiring_clicked(b):
        clear_output(wait=True)
        cur.execute("SELECT * FROM food_inventory WHERE days_until_expiry <= 30;")
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        display(widgets.VBox([random_btn, bycat_btn, expiring_btn, return_btn]))
        display(df)

    def on_return_clicked(b):
        show_main_menu()

    random_btn.on_click(on_random_clicked)
    bycat_btn.on_click(on_bycat_clicked)
    expiring_btn.on_click(on_expiring_clicked)
    return_btn.on_click(on_return_clicked)
    submenu_box = widgets.VBox([random_btn, bycat_btn, expiring_btn, return_btn])
    display(submenu_box)

def show_export_submenu():
    def export_csv_optimized(df):
        print("Preparing CSV export (this may take several minutes)...")
        # Write in chunks to avoid memory issues
        df.to_csv('food_inventory.csv', index=False, chunksize=1000000)

    def on_csv_clicked(b):
        cur.execute("SELECT * FROM food_inventory;")
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        export_csv_optimized(df)

    def export_excel_optimized(df):
        print("Preparing Excel export (this may take several minutes)...")
        df.to_excel('food_inventory.xlsx', index=False, chunksize=1000000)

    def on_excel_clicked(b):
        cur.execute("SELECT * FROM food_inventory;")
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        export_excel_optimized(df)

    def export_json_optimized(df):
        print("Preparing JSON export (this may take several minutes)...")
        df.to_json('food_inventory.json', index=False, chunksize=1000000)

    def on_json_clicked(b):
        cur.execute("SELECT * FROM food_inventory;")
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        
        
    clear_output(wait=True)
    csv_btn = widgets.Button(description='Export to CSV')
    excel_btn = widgets.Button(description='Export to Excel')
    json_btn = widgets.Button(description='Export to JSON')

    csv_btn.on_click(on_csv_clicked)
    excel_btn.on_click(on_excel_clicked)
    json_btn.on_click(on_json_clicked)

    return_btn = widgets.Button(description='Return', button_style='danger')
    def on_return_clicked(b):
        show_main_menu()
    return_btn.on_click(on_return_clicked)

    display(widgets.VBox([csv_btn, excel_btn, json_btn, return_btn]))

def on_export_clicked(b):
    show_export_submenu()

def on_view_sample_clicked(b):
    show_sample_submenu()

def on_exit_clicked(b):
    menu_box.close()

view_btn = widgets.Button(description='View sample')
export_btn = widgets.Button(description='Export')
exit_btn = widgets.Button(description='Exit', button_style='danger')

exit_btn.on_click(on_exit_clicked)
view_btn.on_click(on_view_sample_clicked)
export_btn.on_click(on_export_clicked)
menu_box = widgets.VBox([view_btn, export_btn, exit_btn])

show_main_menu()

Finally, we close the connection to the database:

In [5]:
conn.close()